# MLM Training

In this notebook we'll cover the training process for masked-language modeling (MLM). First we import and initialize everything required.

In [1]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


We'll be using *Meditations* by *Marcus Aurelius* as our training data. The file below has already been cleaned and so no further processing is required (beyond the `split`).

In [2]:
with open('./clean.txt', 'r') as fp:
    text = fp.read().split('\n')

In [3]:
text[:5]

['From my grandfather Verus I learned good morals and the government of my temper.',
 'From the reputation and remembrance of my father, modesty and a manly character.',
 'From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evil thoughts; and further, simplicity in my way of living, far removed from the habits of the rich.',
 'From my great-grandfather, not to have frequented public schools, and to have had good teachers at home, and to know that on such things a man should spend liberally.',
 "From my governor, to be neither of the green nor of the blue party at the games in the Circus, nor a partizan either of the Parmularius or the Scutarius at the gladiators' fights; from him too I learned endurance of labour, and to want little, and to work with my own hands, and not to meddle with other people's affairs, and not to be ready to listen to slander."]

First, we'll tokenize our text.

In [4]:
inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [5]:
inputs

{'input_ids': tensor([[  101,  2013,  2026,  ...,     0,     0,     0],
        [  101,  2013,  1996,  ...,     0,     0,     0],
        [  101,  2013,  2026,  ...,     0,     0,     0],
        ...,
        [  101,  3459,  2185,  ...,     0,     0,     0],
        [  101,  2043, 15223,  ...,     0,     0,     0],
        [  101,  7887,  3288,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

Then we create our *labels* tensor by cloning the *input_ids* tensor.

In [6]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [7]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

Now we mask tokens in the *input_ids* tensor, using the 15% probability we used before - and the **not** a *CLS* or *SEP* token condition. This time, because we have padding tokens we also need to exclude *PAD* tokens (*0* input ids).

In [8]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [9]:
mask_arr

tensor([[False,  True, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False,  True,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

And now we take take the indices of each `True` value, within each individual vector.

In [10]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

/cluster/home/fgonzalez/.virtualenvs/scirex/lib64/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  """


In [11]:
selection[:5]

[[1],
 [4],
 [19, 24, 29, 34, 36, 39, 41],
 [22, 27, 28],
 [2, 4, 6, 12, 13, 17, 18, 27, 28, 37, 44, 48, 51, 52, 55, 64, 67, 87, 89]]

Then apply these indices to each respective row in *input_ids*, assigning each of the values at these indices as *103*.

In [12]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [13]:
inputs.input_ids

tensor([[ 101,  103, 2026,  ...,    0,    0,    0],
        [ 101, 2013, 1996,  ...,    0,    0,    0],
        [ 101, 2013, 2026,  ...,    0,    0,    0],
        ...,
        [ 101, 3459, 2185,  ...,    0,    0,    0],
        [ 101, 2043,  103,  ...,    0,    0,    0],
        [ 101, 7887, 3288,  ...,    0,    0,    0]])

We can see the values *103* have been assigned in the same positions as we found *True* values in the `mask_arr` tensor.

The `inputs` tensors are now ready, and can we can begin setting them up to be fed into our model during training. We create a PyTorch dataset from our data.

In [14]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

Initialize our data using the `MeditationsDataset` class.

In [15]:
dataset = MeditationsDataset(inputs)

And initialize the dataloader, which we'll be using to load our data into the model during training.

In [16]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

Now we can move onto setting up the training loop. First we setup GPU/CPU usage.

In [17]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

Activate the training mode of our model, and initialize our optimizer (Adam with weighted decay - reduces chance of overfitting).

In [18]:
from transformers import AdamW



# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

Now we can move onto the training loop, we'll train for two epochs (change `epochs` to modify this).

In [19]:
from tqdm import tqdm  # for our progress bar

loop = tqdm(loader, leave=True)

In [20]:
batch=next(iter(loader))

/cluster/home/fgonzalez/.virtualenvs/scirex/lib64/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


optim.zero_grad()

In [21]:

# pull all tensor batches required for training
input_ids = batch['input_ids'].to(device)
attention_mask = batch['attention_mask'].to(device)
labels = batch['labels'].to(device)

In [22]:

# process
outputs = model(input_ids, attention_mask=attention_mask,
                labels=labels)

 SIZE:  torch.Size([16, 512])
EMBEDDINGS SIZE:  torch.Size([16, 512, 768])
 embedding_output:  torch.Size([16, 512, 768])
 hidden_states:  torch.Size([16, 512, 768])
 hidden_states:  torch.Size([16, 512, 768])
 hidden_states:  torch.Size([16, 512, 768])
 hidden_states:  torch.Size([16, 512, 768])
 hidden_states:  torch.Size([16, 512, 768])
 hidden_states:  torch.Size([16, 512, 768])
 hidden_states:  torch.Size([16, 512, 768])
 hidden_states:  torch.Size([16, 512, 768])
 hidden_states:  torch.Size([16, 512, 768])
 hidden_states:  torch.Size([16, 512, 768])
 hidden_states:  torch.Size([16, 512, 768])
 hidden_states:  torch.Size([16, 512, 768])
 sequence_output:  torch.Size([16, 512, 768])


In [41]:
outputs['logits'][0].squeeze()

tensor([[ -7.0057,  -6.9682,  -6.9149,  ...,  -6.3493,  -6.1319,  -4.3526],
        [-12.7728, -12.5942, -12.6098,  ..., -11.3728, -10.7302,  -9.2989],
        [ -9.3654,  -8.8331,  -9.0386,  ...,  -8.0218,  -8.8608,  -6.3913],
        ...,
        [ -9.8041,  -9.6077,  -9.1870,  ...,  -9.2433,  -8.9549,  -4.8263],
        [ -6.5214,  -6.5661,  -6.7110,  ...,  -5.4897,  -6.0696,  -4.3684],
        [ -6.1707,  -6.1827,  -6.3590,  ...,  -5.1155,  -5.6416,  -3.7265]],
       device='cuda:0', grad_fn=<SqueezeBackward0>)

In [42]:
top_word = torch.argmax(outputs['logits'][0].squeeze(), dim=1)

In [53]:
input_ids[0][:]

tensor([  101,  2045,  2024,  2176,  4054, 14863,   103,   103,  1997,  1996,
         6020,   103,  2114,  2029, 15223,  2323,  3367,  2022,  7887,  2006,
          103,  3457,  1010,  1998,  2043, 15223,  2038,  2102, 11156,  2068,
         1010, 15223,  2323,  3367, 13387,   103,  2041,   103,  2360,  2006,
         2169,  6686,  2947,   103,  2023,  2245,  2003,  2025,  4072,  1024,
         2023, 12102,  2000,  6033,  2591,  2586,  1024,  2023,  2029,   103,
         2396,  2183,  2000,  2360,  3310,  2025,  2013,  1996,  2613,   103,
         1025,  2005,   103,  2323,  3367,  5136,  2009,  2426,   103,   103,
          103,  1997,  2477,  2005,   103,  2158,  2025,  2000,  3713,  2013,
         2010,  2613,   103,  1012,   103,  1996,  2959,  2003,  2043, 15223,
        21146,   103, 16360,  3217,  6776, 15177,   103,   103,  2005,   103,
         1010,  2005,  2023,  2003,  2019,   103,  1997,  1996,   103,  2099,
         2112,  2306, 14992,  2108,   103, 27267,   103, 21336, 

In [59]:
tokenizer.decode??

In [58]:
tokenizer.decode(input_ids[0][:8], skip_special_tokens=True,)

'there are four principal abe'

In [48]:
tokenizer.decode(input_ids[0], skip_special_tokens=True)

'there are four principal abe of the superior against which thou shouldst be constantly on guard, and when thou hast detected them, thou shouldst wipe out say on each occasion thus this thought is not necessary : this tends to destroy social union : this which art going to say comes not from the real ; for shouldst consider it among of things for man not to speak from his real. the fourth is when thou sha reproach thy for, for this is an of ther part within thee beingpowered yielding to the less honourable and to the perishable part, the, and to its pleasures.'

In [44]:
tokenizer.decode(top_word, skip_special_tokens=True)

'. there are four principal abettingments of the superior parts against which thou shouldst be constantly on thy guard, and when thou hast detected them, thou shouldst wipe them out. say on each occasion thus : this thought is not necessary : this tends to destroy social union : this which thou art going to say comes not from the real world ; for thou shouldst consider it among the manyest of things for a man not to speak from his real self. and the fourth is when thou shalt reproach thy self self for it, for this is an example of the purer part within thee being overpowered and yielding to the less honourable and to the perishable part, the good, and to its own pleasures.. wise them. them : the,,, : thou shouldstst, thy thy guard, that when art has detected detected encountered abe have found thourtirti have them them them..... : : about : : :, is not so necessary per the the all per the good good good encountered not should thou thou them. and... : : : : : : thou is is necessary thou

In [25]:
outputs['logits'].size()

torch.Size([16, 512, 30522])

In [33]:
attention_mask[0]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [34]:
input_ids[0]

tensor([  101,  2045,  2024,  2176,  4054, 14863,   103,   103,  1997,  1996,
         6020,   103,  2114,  2029, 15223,  2323,  3367,  2022,  7887,  2006,
          103,  3457,  1010,  1998,  2043, 15223,  2038,  2102, 11156,  2068,
         1010, 15223,  2323,  3367, 13387,   103,  2041,   103,  2360,  2006,
         2169,  6686,  2947,   103,  2023,  2245,  2003,  2025,  4072,  1024,
         2023, 12102,  2000,  6033,  2591,  2586,  1024,  2023,  2029,   103,
         2396,  2183,  2000,  2360,  3310,  2025,  2013,  1996,  2613,   103,
         1025,  2005,   103,  2323,  3367,  5136,  2009,  2426,   103,   103,
          103,  1997,  2477,  2005,   103,  2158,  2025,  2000,  3713,  2013,
         2010,  2613,   103,  1012,   103,  1996,  2959,  2003,  2043, 15223,
        21146,   103, 16360,  3217,  6776, 15177,   103,   103,  2005,   103,
         1010,  2005,  2023,  2003,  2019,   103,  1997,  1996,   103,  2099,
         2112,  2306, 14992,  2108,   103, 27267,   103, 21336, 

In [32]:
labels[0]

tensor([  101,  2045,  2024,  2176,  4054, 14863, 11335,  9285,  1997,  1996,
         6020,  4513,  2114,  2029, 15223,  2323,  3367,  2022,  7887,  2006,
        15177,  3457,  1010,  1998,  2043, 15223,  2038,  2102, 11156,  2068,
         1010, 15223,  2323,  3367, 13387,  2068,  2041,  1998,  2360,  2006,
         2169,  6686,  2947,  1024,  2023,  2245,  2003,  2025,  4072,  1024,
         2023, 12102,  2000,  6033,  2591,  2586,  1024,  2023,  2029, 15223,
         2396,  2183,  2000,  2360,  3310,  2025,  2013,  1996,  2613,  4301,
         1025,  2005, 15223,  2323,  3367,  5136,  2009,  2426,  1996,  2087,
        18691,  1997,  2477,  2005,  1037,  2158,  2025,  2000,  3713,  2013,
         2010,  2613,  4301,  1012,  2021,  1996,  2959,  2003,  2043, 15223,
        21146,  7096, 16360,  3217,  6776, 15177, 11246,  2546,  2005,  2505,
         1010,  2005,  2023,  2003,  2019,  3350,  1997,  1996,  7746,  2099,
         2112,  2306, 14992,  2108,  2058, 27267,  1998, 21336, 

In [19]:


epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|                                                                                           | 0/32 [00:00<?, ?it/s]<ipython-input-14-6dccb434beb5>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 1: 100%|█████████████████████████████████████████████████████████████| 32/32 [00:11<00:00,  2.86it/s, loss=0.147]


In [25]:
!pip install transformers

You should consider upgrading via the '/cluster/home/fgonzalez/.virtualenvs/scirex/bin/python -m pip install --upgrade pip' command.


And there we go, we've fine-tuned our BERT model using MLM on *Meditations*!